In [43]:
from math import floor

class field(object):
    def __init__(self, elementary_str, contractions, symbol):
        self.elementary_str = elementary_str #Stores list of N symbols for elementary field factors phi and dphi
        self.elementary = []
        self.contractions = contractions #Stores the indices with respect to which successive elementary factors are linked
                                #e.g., [[0,1],[1,0],[0,1]] for phi^i1 dphi_u1^i1 dphi_u1^i2 phi^i2, [[1,1]] for dphi_u^i dphi_u^i. For N 
                                #elementary fields, N-1 links.
        self.symbol = symbol
        self.generated_terms = []
    def __str__(self):
        return 'elementary_str: ' + str(self.elementary_str) + '\n' \
            + 'contractions: ' + str(self.contractions) + '\n' \
            + 'symbol: ' + str(self.symbol) + '\n' \
            + 'massDim: ' + str(self.massDim()) + '\n' \
            + 'LORrank: ' + str(self.LORrank()) + '\n' \
            + 'ONrank: ' + str(self.ONrank()) 
    
    #Define get and set functions
    def get_elementary_str(self):
        return self.elementary_str
    def get_elementary(self):
        return self.elementary
    def get_contractions(self):
        return self.contractions
    def get_symbol(self):
        return self.symbol
    
    def set_elementary_str(self, elementary_str):
        self.elementary_str = elementary_str
    def set_elementary(self): #should run this to set list of constituent fields
        for symbol in self.elementary_str:
            if symbol == 'P':
                field1 = field(['P'],[], 'P')
                self.elementary.append(field1)
            elif symbol == 'DP':
                field1_deriv = field(['DP'],[], 'DP')
                self.elementary.append(field1_deriv)
            else:
                print('Error: Symbol must be P or DP.')
    def set_contractions(self, contractions):
        self.contractions = contractions
    def set_symbol(self, symbol):
        self.symbol = symbol
    
    
    
    def massDim(self):
        
        counter = 0
        
        for field in self.get_elementary():
            if field.get_symbol() == 'P':
                counter = counter + 1
            elif field.get_symbol() == 'DP':
                counter = counter + 2
        
        return counter                       

    def LORrank(self):
        '''
        To get LORrank Count number of Lorentz contractions (counter) in contractions. Multiply by 2. Subtract this from 
        the number of Lorentz indices appearing in elementary - namely, the number of 'DP' fields. 
        '''
        counter1 = 0
        for pair in self.get_contractions():
            if pair[0] == 1:
                counter1 = counter1 + 1
            else:
                pass
            
        counter2 = 0
        for field in self.get_elementary():
            if field.get_symbol() == 'DP':
                counter2 = counter2 + 1
            else:
                pass     
        #print("counter1: " + str(counter1))
        #print("counter2: " + str(counter2))
        
        rank = counter2 - 2*counter1
        
        return rank
    
    
    def ONrank(self):
        
        counter1 = 0
        
        #Number of contractions is the number of time a '1' appears in the second index of pairs in contractions
        for pair in self.get_contractions():
            if pair[1] == 1:
                counter1 = counter1 + 1
            else:
                pass
        
        #Number of ON indices is just the number of fields in self.elementary
        counter2 = len(self.get_elementary_str())
            
        rank = counter2 - 2*counter1
        
        return rank
    
    
    def contract(self, other, contraction, symbol_prod):
        '''
        other: elementary field, P or DP
        contraction: pair of integers - [1,0] for contraction in LOR, [0,1] for contraction in ON, [1,1] for both
        symbol_prod: string
        '''
        
        #print("contraction: " + str(contraction))
        
        if contraction == [1,0]:
            contractions_prod = self.get_contractions() + [[1,0]]
        elif contraction == [0,1]:
            contractions_prod = self.get_contractions() + [[0,1]]
        elif contraction == [1,1]:
            contractions_prod = self.get_contractions() + [[1,1]]
        else:
            print("Error: contractions must be [1,0], [0,1], or [1,1]")
            
        
        elementary_str_prod = self.get_elementary_str() + other.get_elementary_str()
        
        prod = field(elementary_str_prod, contractions_prod, symbol_prod)
        
        prod.set_elementary()
        
        return prod
    
    
    def multiply(self, other, symbol_prod=''):
        '''
        Multiplies without contraction
        INPUT:
        self: field object
        other: field object
        symbol_prod: symbol for new field obj output
        
        OUTPUT:
        prod: field object, concatenating self and other, by inserting [0,0] in contractions
        
        '''
        if (not self.get_elementary_str()) or (not other.get_elementary_str()):
            print("one factor is const")
            contractions_prod = self.get_contractions() + other.get_contractions()
        else:
            contractions_prod = self.get_contractions() + [[0,0]] + other.get_contractions()
        
        elementary_str_prod = self.get_elementary_str() + other.get_elementary_str()
        
        prod = field(elementary_str_prod, contractions_prod, symbol_prod)
        
        prod.set_elementary()
        
        return prod
    
    
    
    def generate_connected(self, d):
        '''
        INPUT
        d: non-negative integer, mass dimension up to which to generate field terms
        
        OUTPUT
        terms: list of field terms of mass dimension no larger than d.  
        '''
        
        const = field([],[],'1')
        
        terms = [const]
        
        #print("in generate_connected")
        
        if d >= 2:
            #print("in d>=2")
            field1 = field(['P'],[],'P')
            field2 = field(['P'],[],'P')
            prod = field.contract(field1, field2, [0,1], 'PP')
            terms.append(prod)
        if d >= 4:
            deriv1 = field(['DP'],[],'DP')
            deriv2 = field(['DP'],[],'DP')
            prod = field.contract(deriv1, deriv2, [1,1], 'DPDP')
            terms.append(prod)
        
        field1 = field(['P'],[],'P')
        deriv1 = field(['DP'],[],'DP')
        
        if d < 4:
            return terms
    
        
        root = field.contract(field1, deriv1, [0,1], 'PDP')
        
        while root.massDim() <= d:
            #print("")
            #print("root: ")
            #print(root)
            #print("")
            if root.LORrank() != 0:
                #print("in root.LORrank()!=0")
                deriv1 = field(['DP'],[],'DP')
                root = field.contract(root, deriv1, [1,0], '')
                #print("")
                #print("root: ")
                #print(root)
                #print("")
            
            if root.ONrank() != 0:
                #print("in root.ONrank()!=0")
                field1 = field(['P'],[],'P')
                deriv1 = field(['DP'],[],'DP')
                
                new = field.contract(root, field1, [0,1], '')
                if new.massDim() <= d and new.LORrank() == 0 and new.ONrank() == 0:
                    #print("in new term added")
                    terms.append(new)
                
                root = field.contract(root, deriv1, [0,1], '')
                #print("")
                #print("root: ")
                #print(root)
                #print("")    
        
        return terms
    
    
    
    def generate(self, D):
        print("In generate, D = " + str(D))
        if D < 2:
            return [field([], [], '')], [field([], [], '')] 
        if D == 2:
            const = field([], [], '')
            PP = field(['P','P'],[[0,1]],'PP')
            return [const, PP], [const, PP] #return generated terms and cache for higher D's
        else:
           
            
            #check if cached terms not yet admitted into generated terms have dim <= D.  
            #generated = self.generate(D-1)[0]
            generated = []
            cache = self.generate(D-1)[1]
            #check = list(set(cache_prev) - set(generated)
            
    
            #Generate connected terms 
            connected_terms_D = field.generate_connected(self, D)
            print("connected_terms_D:")
            for term in connected_terms_D:
                print(term)
            
            #Multiply all elements in cache by new set of connected terms
            for term1 in cache:
                for term2 in connected_terms_D:
                    prod = field.multiply(term1, term2)
                    #print("prod: ")
                    #print(prod)
                    cache.append(prod)
                    
            #Filter out elements of cache that are within mass threshold
            for i in range(len(cache)):
                if cache[i].massDim() <= D:
                    generated.append(cache[i])
            '''
            print("")
            print("")
            print("Checking cache, D = " + str(D))
            for term in cache:
                print("")
                print(term)
            print("")
            print("")
            '''
            
                    
            
            return generated, cache
                
                
            
        

In [44]:
A = field([],[],'')

In [45]:
A.generate(10)

In generate, D = 10
In generate, D = 9
In generate, D = 8
In generate, D = 7
In generate, D = 6
In generate, D = 5
In generate, D = 4
In generate, D = 3
In generate, D = 2
connected_terms_D:
elementary_str: []
contractions: []
symbol: 1
massDim: 0
LORrank: 0
ONrank: 0
elementary_str: ['P', 'P']
contractions: [[0, 1]]
symbol: PP
massDim: 2
LORrank: 0
ONrank: 0
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is cons

one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const


one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const


one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const


one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const


...

one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const


one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const


one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const


one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const
one factor is const


KeyboardInterrupt: 

In [ ]:
print(A.generate(10)[0][1])

In [46]:
B = field([],[],'')

In [47]:
mylist = [A, B]

In [48]:
mylist

[<__main__.field at 0x1f4bd5160>, <__main__.field at 0x1373007b8>]

In [49]:
list(set(mylist))

[<__main__.field at 0x1373007b8>, <__main__.field at 0x1f4bd5160>]

In [50]:
C = list(set(mylist))

In [51]:
C

[<__main__.field at 0x1373007b8>, <__main__.field at 0x1f4bd5160>]

In [52]:
print({A.generate_connected(10)[i] for i in range(len(A.generate_connected(10)))})

{<__main__.field object at 0x161dc7e48>, <__main__.field object at 0x161df4278>, <__main__.field object at 0x161ad5ac8>, <__main__.field object at 0x1fffdcf98>, <__main__.field object at 0x161df4ba8>}


In [57]:
print(A.generate_connected(10)[4] )

elementary_str: ['P', 'DP', 'DP', 'DP', 'DP', 'P']
contractions: [[0, 1], [1, 0], [0, 1], [1, 0], [0, 1]]
symbol: 
massDim: 10
LORrank: 0
ONrank: 0
